## Install pyTigerGraph

In [ ]:
# Setup
!pip install -U pyTigerGraph

## Add Imports and Establish Initial Connection

In [ ]:
# Imports
import json
import pandas as pd

from google.colab import drive
import pyTigerGraph as tg

drive.mount('/gdrive')

# Connection parameters
hostName = ''
userName = ''
password = ''
conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password)

conn.graphname='TigyoreGraph'
secret = conn.createSecret()
authToken = conn.getToken(secret)
authToken = authToken[0]
conn = tg.TigerGraphConnection(host=hostName, graphname='TigyoreGraph', username=userName, password=password, apiToken=authToken)

print('Connected')

In [ ]:
!ls "/gdrive/My Drive/Tigyore/data/nyc_trips/nyc_trips_data"

In [ ]:
!head -n 200 "/gdrive/My Drive/Tigyore/data/nyc_trips/nyc_trips_data/trips_processed.csv"

## Create Loading Jobs

## Trips

### Create Load Data Job

Here it's important to note that the `$0`, `$1` values line up with the columns of your data.

In [ ]:
results = conn.gsql('''
USE GRAPH TigyoreGraph
SHOW VERTEX Trip
SHOW VERTEX BusStop
SHOW VERTEX TrainStation
SHOW VERTEX BusService
SHOW VERTEX TrainService
SHOW VERTEX Location
SHOW EDGE waypoint
SHOW EDGE utilise
''')

print(results)

In [ ]:
trip_results = conn.gsql('''
  USE GRAPH TigyoreGraph
  BEGIN
  CREATE LOADING JOB load_trips FOR GRAPH TigyoreGraph {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO VERTEX Trip VALUES($0, $8, $9) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  }
  END
  ''')

location_results = conn.gsql('''
  USE GRAPH TigyoreGraph
  BEGIN
  CREATE LOADING JOB load_locations FOR GRAPH TigyoreGraph {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO VERTEX Location VALUES($12, $7, $5, $6) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  }
  END
  ''')

waypoint_train_results = conn.gsql('''
  USE GRAPH TigyoreGraph
  BEGIN
  CREATE LOADING JOB load_waypoint_train FOR GRAPH TigyoreGraph {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO EDGE waypoint VALUES($0 Trip, $8 TrainStation, $1, 1) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  }
  END
  ''')

waypoint_bus_results = conn.gsql('''
  USE GRAPH TigyoreGraph
  BEGIN
  CREATE LOADING JOB load_waypoint_bus FOR GRAPH TigyoreGraph {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO EDGE waypoint VALUES($0 Trip, $8 BusStop, $1, 1) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  }
  END
  ''')

waypoint_other_results = conn.gsql('''
  USE GRAPH TigyoreGraph
  BEGIN
  CREATE LOADING JOB load_waypoint_other FOR GRAPH TigyoreGraph {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO EDGE waypoint VALUES($0 Trip, $12 Location, $1, 0) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  }
  END
  ''')

utilise_train_results = conn.gsql('''
  USE GRAPH TigyoreGraph
  BEGIN
  CREATE LOADING JOB load_utilise_train FOR GRAPH TigyoreGraph {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO EDGE utilise VALUES($0 Trip, $7 TrainService, $1, $1) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  }
  END
  ''')

utilise_bus_results = conn.gsql('''
  USE GRAPH TigyoreGraph
  BEGIN
  CREATE LOADING JOB load_utilise_bus FOR GRAPH TigyoreGraph {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO EDGE utilise VALUES($0 Trip, $7 BusService, $1, $1) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  }
  END
  ''')

print(trip_results, location_results, waypoint_train_results, waypoint_bus_results, waypoint_other_results, utilise_train_results, utilise_bus_results)

In [ ]:
results = conn.gsql('''
USE GRAPH TigyoreGraph
SHOW JOB load_trips
SHOW JOB load_locations
SHOW JOB load_waypoint_train
SHOW JOB load_waypoint_bus
SHOW JOB load_waypoint_other
SHOW JOB load_utilise_train
SHOW JOB load_utilise_bus
''')

print(results)

### Load Data

In [ ]:
data_file = '/gdrive/My Drive/Tigyore/data/nyc_trips/nyc_trips_data/trips_processed.csv'
results = conn.uploadFile(data_file, fileTag='MyDataSource', jobName='load_trips')
print(json.dumps(results, indent=2))

In [ ]:
data_file = '/gdrive/My Drive/Tigyore/data/nyc_trips/nyc_trips_mapped/nyc_trips_subway.csv'

results = conn.uploadFile(data_file, fileTag='MyDataSource', jobName='load_waypoint_train')
print(json.dumps(results, indent=2))

results = conn.uploadFile(data_file, fileTag='MyDataSource', jobName='load_utilise_train')
print(json.dumps(results, indent=2))

In [ ]:
data_file = '/gdrive/My Drive/Tigyore/data/nyc_trips/nyc_trips_mapped/nyc_trips_bus.csv'

results = conn.uploadFile(data_file, fileTag='MyDataSource', jobName='load_waypoint_bus')
print(json.dumps(results, indent=2))

results = conn.uploadFile(data_file, fileTag='MyDataSource', jobName='load_utilise_bus')
print(json.dumps(results, indent=2))

In [ ]:
data_file = '/gdrive/My Drive/Tigyore/data/nyc_trips/nyc_trips_mapped/nyc_trips_other.csv'

results = conn.uploadFile(data_file, fileTag='MyDataSource', jobName='load_location')
print(json.dumps(results, indent=2))

results = conn.uploadFile(data_file, fileTag='MyDataSource', jobName='load_waypoint_other')
print(json.dumps(results, indent=2))